In [1]:
import pandas as pd
import numpy as np
import re
from mecab import MeCab
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random

In [2]:
mecab = MeCab()

In [3]:
df = pd.read_csv('dementia_fix.csv', sep=',', encoding='utf-8', index_col=0)

In [4]:
df

,question,intention,answer
0,"알츠하이머병의 원인으로 매일 소주를 섭취하는 것이 언급되고 있는데, 이에 대한 근거...",원인,"알츠하이머병의 정확한 원인은 아직 밝혀지지 않았지만, 연구들이 알츠하이머병의 발병 ..."
1,알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요?,원인,알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다.알츠하이머병은 아직 완전히...
2,알츠하이머병의 발생 원인에 대한 연구나 발견이 진행 중인가요?,원인,"알츠하이머병은 치매를 일으키는 가장 흔한 퇴행성 뇌질환으로, 1907년 독일 의사 ..."
3,알츠하이머병의 발병과 관련하여 뇌의 노화로 인한 증상과 원인을 알려주세요.,원인,알츠하이머병은 현재까지 그 발병 원인에 대한 완벽한 해명은 아직 이루어지지 않았습니...
4,알츠하이머병의 원인과 관련된 연구 결과가 있을까요? 알려주세요.,원인,"알츠하이머병은 복잡한 질환으로, 아직도 원인이 완전히 밝혀진 것은 아닙니다. 그러나..."
...,...,...,...
6618,치매 치료에는 어떤 운동이나 작업이 효과적일까요?,치료,"치매는 노인들에게 주로 발생하는 뇌질환으로, 원인과 치료 방법은 아직 완전히 밝혀진..."
6619,치매 치료의 결과와 과정을 상세히 설명해주세요. 치매 치료의 효과는 어떻게 나타날까요?,치료,"치매는 일상 생활을 수행하는 능력을 심각하게 손상시키는 질환으로, 후천성 치매와 노..."
6620,치매를 치료하기 위해 어떤 치료 방법들이 효과적일까요?,치료,"치매는 노화로 인해 기억력과 지능을 점차적으로 잃는 질병으로, 알츠하이머병이 주요한..."
6621,치매 치료를 위해 어떤 약물이 사용될 수 있을까요?,치료,알츠하이머병은 뇌에 변화가 생겨서 인지 기능에 장애가 생기는 신경퇴행성 질환입니다....


In [5]:
q_data = df['question']
a_data = df['answer']

In [6]:
# korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'
# clean = re.compile(korean_pattern)
# a = ' '.join(q_data.tolist() + a_data.tolist())
# a = a.lower()
# clean_result = clean.sub("", a)
# morphs = mecab.morphs(clean_result)

In [7]:
# word_to_idx = {'<PAD>' : 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}
# word_to_idx.update({word: idx + 4 for idx, word in enumerate(set(morphs))})

In [8]:
# idx_to_word = {word: idx for idx, word in word_to_idx.items()}

In [9]:
with open('word_to_idx.pkl', 'rb') as f:
    word_to_idx = pickle.load(f)
idx_to_word = {word: idx for idx, word in word_to_idx.items()}

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
hidden_size = 128
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2
UNK_TOKEN = 3
MAX_LENGTH = 300

In [12]:
def indiceFromSentence(vocab, sentence):
    return [vocab.get(word, vocab['<UNK>']) for word in mecab.morphs(sentence)]

def tensorFromSentence(vocab, sentence):
    indice = indiceFromSentence(vocab, sentence)
    indice.append(EOS_TOKEN) 
    return torch.tensor(indice, dtype=torch.long, device=device).view(-1, 1)

In [13]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2)
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden
    
    def initHidden(self):
        return(torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))

In [14]:
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output[0])
        return output, hidden
    
    def initHidden(self):
        return(torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))

In [15]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        
    decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
    decoder_hidden = encoder_hidden
    
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        loss += criterion(decoder_output, target_tensor[di])
        
        if decoder_input.item() == EOS_TOKEN:
            break
        
    loss.backward() # 역전파 
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length
    
    

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000):
    print_loss_total = 0
    
    for iter in range(1, n_iters+1):
        training_pair = random.choice(pairs) # input - target pair
        input_tensor = tensorFromSentence(word_to_idx, training_pair[0]).to(device)
        target_tensor = tensorFromSentence(word_to_idx, training_pair[1]).to(device)
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        
        if iter % print_every == 0:
            print_lost_avg = print_loss_total / print_every
            print(f'Iteration : {iter}, Loss : {print_lost_avg: .4f}')
            print_loss_total = 0

In [17]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(word_to_idx, sentence).to(device)
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()
        encoder_hidden = tuple([e.to(device) for e in encoder_hidden])
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            
        decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = [] # output sentence
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_TOKEN:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(idx_to_word[topi.item()]) # 최종 아웃풋의 index
            
            decoder_input = topi.squeeze().detach()
        print(decoded_words)
        return ' '.join(decoded_words)

In [18]:
def chat(encoder, decoder, max_length=MAX_LENGTH):
    print("Let's chat (type 'bye' to exit)")
    while True:
        input_sentence = input(">>")
        if input_sentence == 'bye':
            break
        output_sentence = evaluate(encoder, decoder, input_sentence)
        print('<', output_sentence)

In [19]:
vocab_size = len(word_to_idx)

In [20]:
encoder = EncoderLSTM(vocab_size, hidden_size).to(device)
decoder = DecoderLSTM(hidden_size, vocab_size).to(device)
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [21]:
pairs = [list(x) for x in zip(q_data, a_data)]

In [22]:
pairs[1]

['알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요?',
 '알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다.알츠하이머병은 아직 완전히 이해되지 않았지만, 연구 결과에 따르면 유전적인 요소와 다양한 환경적인 요인이 이 질환을 일으키는 역할을 한다고 알려져 있습니다. 특히, 아밀로이드 베타 단백질의 비정상적인 축적이 알츠하이머병과 관련이 있는 것으로 알려져 있습니다. 이 외에도 나이, 노화, 고혈압, 당뇨병, 그리고 흡연 등과 같은 다른 요인들도 알츠하이머병 발병과 연관성이 있을 수 있습니다.더 많은 연구와 조사를 통해 알츠하이머병의 원인을 파악하고 예방 방법을 개발할 필요가 있습니다.']

In [ ]:
encoder.train()
decoder.train()

DecoderLSTM(
  (embedding): Embedding(5030, 128)
  (lstm): LSTM(128, 128, num_layers=2)
  (out): Linear(in_features=128, out_features=5030, bias=True)
)

: 

In [ ]:
trainIters(encoder, decoder, 1000, 100)

tensor([[  76],
        [  25],
        [1293],
        [ 539],
        [  12],
        [  13],
        [  67],
        [  22],
        [  70],
        [  89],
        [  90],
        [  28],
        [ 160],
        [ 101],
        [  19],
        [  58],
        [  40],
        [   2]], device='cuda:0')
tensor([[850],
        [707],
        [162],
        [ 37],
        [ 38],
        [124],
        [ 62],
        [ 74],
        [ 55],
        [155],
        [156],
        [163],
        [ 25],
        [174],
        [175],
        [ 40],
        [  2]], device='cuda:0')
tensor([[476],
        [ 80],
        [ 76],
        [  5],
        [162],
        [ 37],
        [ 38],
        [124],
        [ 62],
        [ 74],
        [322],
        [ 10],
        [402],
        [165],
        [ 40],
        [  2]], device='cuda:0')
tensor([[ 76],
        [  5],
        [ 60],
        [ 55],
        [ 48],
        [329],
        [263],
        [101],
        [ 19],
        [ 13],
        [ 55]

In [ ]:
encoder.eval()
decoder.eval()

DecoderLSTM(
  (embedding): Embedding(5030, 128)
  (lstm): LSTM(128, 128, num_layers=2)
  (out): Linear(in_features=128, out_features=5030, bias=True)
)

In [ ]:
question = "알츠하이머 원인에 대해 알려줘."
output_sentence = evaluate(encoder, decoder, question)
print(output_sentence)

['우울증', '은', '은', '은', '의', '의', '의', '는', '는', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.

In [ ]:
chat(encoder, decoder)

Let's chat (type 'bye' to exit)
